In [ ]:
# !pip install av clip_client

In [ ]:
from docarray import Document,DocumentArray
video_uri = "https://archive.org/download/santa-clause-conquers-the-martians/Santa%20Clause%20Conquers%20The%20Martians.ia.mp4"
video_data = Document(uri=video_uri).load_uri_to_video_tensor()

In [ ]:
print(video_data.tensor.shape)

In [ ]:
video_data.display()

In [ ]:
import numpy
Document(tensor=numpy.rot90(video_data.tensor[100], -1)).display()

In [ ]:
clip_data = video_data.tensor[300:600]
Document(tensor=clip_data).save_video_tensor_to_file("clip.mp4")
Document(uri="clip.mp4").display()

In [ ]:
print(video_data.tags)

In [ ]:
from docarray import Document, DocumentArray
from numpy import rot90

keyframe_indices = video_data.tags['keyframe_indices']
keyframes = DocumentArray()
for idx in range(0, len(keyframe_indices) - 1):
	keyframe_number = keyframe_indices[idx]
	keyframe_tensor = rot90(video_data.tensor[keyframe_number], -1)
	clip_indices = {
        'start': str(keyframe_number),
        'end': str(keyframe_indices[idx + 1]),
    }
	keyframe = Document(tags=clip_indices, tensor=keyframe_tensor)
	keyframes.append(keyframe)

In [ ]:
from docarray import Document, DocumentArray
from clip_client import Client

server_url = "grpcs://api.clip.jina.ai:2096"

# substitute your own token in the line below!
jina_auth_token = "ef8dd06701869bc93d1849de103617ae"

client = Client(server_url, credential={"Authorization": jina_auth_token})

In [ ]:
query = Document(text="space shuttle launch", matches=keyframes)
ranked_result = client.rank([query])[0]

In [ ]:
print(ranked_result.matches[0].tags)

In [ ]:
match = ranked_result.matches[0]
start_frame = int(match.tags['start'])
end_frame = int(match.tags['end'])
clip_data = video_data.tensor[start_frame:end_frame] 
Document(tensor=clip_data).save_video_tensor_to_file("match.mp4")
Document(uri="match.mp4").display()